# Lesson 4 - Exercise 3: Create Materialized View for Fare Analytics

## Learning Objectives

1. Understand when and why to use materialized views
2. Write CREATE MATERIALIZED VIEW syntax
3. Implement REFRESH logic
4. Validate materialized view results

## Prerequisites

- `aws_config.py` with your credentials
- `data/van_transit_trips_postgres.csv` data file
- Completed Exercise 2 (or this notebook will set up tables automatically)

In [1]:
# ========= SETUP: Imports, Config, and Functions (Run this cell first!) =========

import os
import time
from datetime import datetime
from typing import Dict, Any, List

import pandas as pd
import numpy as np
import boto3

# Load AWS credentials from aws_config.py
import aws_config

# Configuration
AWS_REGION = os.getenv('AWS_REGION')
REDSHIFT_DATABASE = os.getenv('REDSHIFT_DATABASE')
REDSHIFT_WORKGROUP = os.getenv('REDSHIFT_WORKGROUP')
REDSHIFT_SECRET_ARN = os.getenv('REDSHIFT_SECRET_ARN', None)  # Optional

print("Configuration:")
print(f"   Region: {AWS_REGION}")
print(f"   Database: {REDSHIFT_DATABASE}")
print(f"   Workgroup: {REDSHIFT_WORKGROUP}")

# Redshift Data API client
session_boto = boto3.Session(region_name=AWS_REGION)
rsd = session_boto.client("redshift-data", region_name=AWS_REGION)


def _rs_kwargs() -> Dict[str, Any]:
    """Build connection arguments for Redshift Data API."""
    base = dict(Database=REDSHIFT_DATABASE)
    if REDSHIFT_WORKGROUP:
        base["WorkgroupName"] = REDSHIFT_WORKGROUP
        if REDSHIFT_SECRET_ARN:
            base["SecretArn"] = REDSHIFT_SECRET_ARN
    return base


def rs_exec(sql: str, return_results=False, timeout_s=900):
    """Execute SQL on Redshift via the Data API."""
    sql = sql.strip()
    if not sql:
        return None
    
    kwargs = _rs_kwargs()
    kwargs["Sql"] = sql
    
    sid = rsd.execute_statement(**kwargs)["Id"]
    
    start = time.time()
    while True:
        d = rsd.describe_statement(Id=sid)
        if d["Status"] in ("FINISHED", "FAILED", "ABORTED"):
            break
        if time.time() - start > timeout_s:
            raise TimeoutError("Redshift statement timeout")
        time.sleep(0.5)
    
    if d["Status"] != "FINISHED":
        raise RuntimeError(f"Redshift SQL failed: {d.get('Error')}\n---\n{sql}")
    
    if return_results or sql.strip().lower().startswith("select"):
        out, next_token = [], None
        while True:
            args = dict(Id=sid)
            if next_token:
                args["NextToken"] = next_token
            r = rsd.get_statement_result(**args)
            cols = [c["name"] for c in r["ColumnMetadata"]]
            for rec in r["Records"]:
                row = [next(iter(cell.values())) for cell in rec]
                out.append(dict(zip(cols, row)))
            next_token = r.get("NextToken")
            if not next_token:
                break
        return out
    
    return None


print("\nFunctions defined: rs_exec()")
print("Setup complete!")

Configuration:
   Region: us-east-1
   Database: dev
   Workgroup: udacity-dwh-wg

Functions defined: rs_exec()
Setup complete!


---
## Check Prerequisites

This exercise requires the fact table from Exercise 2. If it doesn't exist, run Exercise 2 first.

In [2]:
# ========= Check if prerequisite tables exist =========

print("Checking prerequisite tables...")
print("=" * 60)

try:
    result = rs_exec("SELECT COUNT(*) AS cnt FROM public.dw_fact_trips;")
    print(f"   dw_fact_trips: {result[0]['cnt']} rows")
    
    result = rs_exec("SELECT COUNT(*) AS cnt FROM public.dw_dim_fare_class;")
    print(f"   dw_dim_fare_class: {result[0]['cnt']} rows")
    
    print("\n   Prerequisites met! Ready to create materialized view.")
except Exception as e:
    print(f"   ERROR: {e}")
    print("\n   Please run Exercise 2 first to create the required tables.")

Checking prerequisite tables...
   dw_fact_trips: 8883 rows
   dw_dim_fare_class: 5 rows

   Prerequisites met! Ready to create materialized view.


---
## Step 1: Create the Materialized View

Materialized views pre-compute and store query results for fast access.

In [3]:
# ========= STEP 1: Create Materialized View =========

print("Creating materialized view...")
print("=" * 60)

# Drop if exists
rs_exec("DROP MATERIALIZED VIEW IF EXISTS public.dw_mv_fare_summary;")
print("   Dropped existing view (if any)")

# Create the materialized view
rs_exec("""
CREATE MATERIALIZED VIEW public.dw_mv_fare_summary AS
SELECT 
    dfc.fare_class,
    COUNT(*) AS trip_count,
    SUM(f.total_fare_cad) AS total_revenue_cad,
    AVG(f.total_fare_cad) AS avg_fare_cad,
    MIN(f.total_fare_cad) AS min_fare_cad,
    MAX(f.total_fare_cad) AS max_fare_cad,
    SUM(f.discount_amount_cad) AS total_discounts_cad,
    AVG(f.discount_rate) AS avg_discount_rate,
    AVG(f.distance_km) AS avg_distance_km,
    SUM(f.distance_km) AS total_distance_km,
    AVG(f.transfers)::DECIMAL(5,2) AS avg_transfers,
    SUM(CASE WHEN f.on_time_arrival THEN 1 ELSE 0 END) AS on_time_trips,
    (SUM(CASE WHEN f.on_time_arrival THEN 1 ELSE 0 END)::DECIMAL / NULLIF(COUNT(*), 0) * 100)::DECIMAL(5,2) AS on_time_pct
FROM public.dw_fact_trips f
JOIN public.dw_dim_fare_class dfc ON f.fare_class_sk = dfc.fare_class_sk
GROUP BY dfc.fare_class;
""")

print("   Materialized view created!")

# Verify
result = rs_exec("SELECT COUNT(*) AS cnt FROM public.dw_mv_fare_summary;")
print(f"   Rows in view: {result[0]['cnt']}")

Creating materialized view...
   Dropped existing view (if any)
   Materialized view created!
   Rows in view: 5


---
## Step 2: Refresh the Materialized View

After data changes, refresh the view to update results.

In [4]:
# ========= STEP 2: Refresh Materialized View =========

print("Refreshing materialized view...")
print("=" * 60)

rs_exec("REFRESH MATERIALIZED VIEW public.dw_mv_fare_summary;")
print("   Materialized view refreshed!")

Refreshing materialized view...
   Materialized view refreshed!


---
## Step 3: Query the Materialized View

In [5]:
# ========= STEP 3: Query Materialized View =========

print("Fare Summary by Fare Class:")
print("=" * 60)

results = rs_exec("""
SELECT 
    fare_class,
    trip_count,
    ROUND(total_revenue_cad, 2) AS total_revenue,
    ROUND(avg_fare_cad, 2) AS avg_fare,
    ROUND(avg_distance_km, 2) AS avg_distance,
    on_time_pct
FROM public.dw_mv_fare_summary
ORDER BY total_revenue_cad DESC;
""")

display(pd.DataFrame(results))

Fare Summary by Fare Class:


,fare_class,trip_count,total_revenue,avg_fare,avg_distance,on_time_pct
0,adult,5483,18826.77,3.43,10.73,88.60
1,youth,1468,3503.22,2.38,11.06,88.14
2,senior,1093,2565.20,2.34,10.88,87.64
3,low_income,518,1041.74,2.01,11.03,87.64
4,tourist,321,965.68,3.00,10.76,89.09


In [6]:
# ========= Additional Analytics =========

print("Additional Analytics")
print("=" * 60)

print("\n1. Revenue Share by Fare Class:")
results = rs_exec("""
SELECT 
    fare_class,
    ROUND(total_revenue_cad, 2) AS revenue,
    ROUND(total_revenue_cad / SUM(total_revenue_cad) OVER() * 100, 1) AS pct_of_total
FROM public.dw_mv_fare_summary
ORDER BY total_revenue_cad DESC;
""")
display(pd.DataFrame(results))

print("\n2. Discount Analysis:")
results = rs_exec("""
SELECT 
    fare_class,
    trip_count,
    ROUND(total_discounts_cad, 2) AS total_discounts,
    ROUND(avg_discount_rate * 100, 1) AS avg_discount_pct
FROM public.dw_mv_fare_summary
ORDER BY total_discounts_cad DESC;
""")
display(pd.DataFrame(results))

Additional Analytics

1. Revenue Share by Fare Class:


,fare_class,revenue,pct_of_total
0,adult,18826.77,70.0
1,youth,3503.22,13.0
2,senior,2565.20,9.5
3,low_income,1041.74,3.9
4,tourist,965.68,3.6



2. Discount Analysis:


,fare_class,trip_count,total_discounts,avg_discount_pct
0,youth,1468,1582.13,32.0
1,senior,1093,1293.35,35.0
2,low_income,518,687.98,40.0
3,tourist,321,107.49,10.0
4,adult,5483,0.00,0.0


---
## Step 4: Validate Materialized View

In [7]:
# ========= STEP 4: Validate MV matches base tables =========

print("Validating materialized view against base tables...")
print("=" * 60)

# Query MV
mv_result = rs_exec("""
SELECT SUM(trip_count) AS total_trips, ROUND(SUM(total_revenue_cad), 2) AS total_revenue
FROM public.dw_mv_fare_summary;
""")

# Query base tables directly
base_result = rs_exec("""
SELECT COUNT(*) AS total_trips, ROUND(SUM(total_fare_cad), 2) AS total_revenue
FROM public.dw_fact_trips f
JOIN public.dw_dim_fare_class dfc ON f.fare_class_sk = dfc.fare_class_sk;
""")

print(f"\nMaterialized View: {mv_result[0]['total_trips']} trips, ${mv_result[0]['total_revenue']} revenue")
print(f"Base Tables:       {base_result[0]['total_trips']} trips, ${base_result[0]['total_revenue']} revenue")

if str(mv_result[0]['total_trips']) == str(base_result[0]['total_trips']):
    print("\nVALIDATION PASSED!")
else:
    print("\nWARNING: Mismatch - consider refreshing the view.")

Validating materialized view against base tables...

Materialized View: 8883 trips, $26902.61 revenue
Base Tables:       8883 trips, $26902.61 revenue

VALIDATION PASSED!


---
## Summary

### Lesson 4 - Exercise 3 Complete

You learned:

1. **When to use materialized views** (frequent, expensive queries)
2. **CREATE MATERIALIZED VIEW** syntax
3. **REFRESH MATERIALIZED VIEW** to update data
4. **How to validate MV results** against base tables
5. **Best practices** for MV maintenance